In [1]:
%cd ../..

/home/ivanstefanov/Repositories/used-cars-price-prediction


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [5]:
from price_predictor.data_preparation.feature_engineering import add_age_feature
from price_predictor.data_preparation.pick_or_drop_features import (
    pick_by_datatypes,
    drop_anonymous_features,
    drop_categorical_features,
    drop_by_name
)

from price_predictor.trainers.train_linear_model import train_linear_regression
from price_predictor.trainers.train_ensamble_model import train_random_forest, train_ada_boost
from price_predictor.evaluate import evaluate_model

In [6]:
data = pd.read_csv('data/cars.csv')
data.dropna(inplace=True)

## Try numerical with car age, not year_produced

In [7]:
df = add_age_feature(data)

df = drop_anonymous_features(df)
df = drop_categorical_features(df)
df = drop_by_name(df, 'year_produced')
df

,odometer_value,engine_has_gas,engine_capacity,has_warranty,price_usd,is_exchangeable,number_of_photos,up_counter,duration_listed,age
0,190000,False,2.5,False,10900.00,False,9,13,16,9
1,290000,False,3.0,False,5000.00,True,12,54,83,17
2,402000,False,2.5,False,2800.00,True,4,72,151,18
3,10000,False,3.0,False,9999.00,True,9,42,86,20
4,280000,False,2.5,False,2134.11,True,14,7,7,18
...,...,...,...,...,...,...,...,...,...,...
38526,290000,False,3.5,False,2750.00,True,5,85,301,19
38527,321000,False,2.2,False,4800.00,True,4,20,317,15
38528,777957,False,3.5,False,4300.00,False,3,63,369,19
38529,20000,False,2.0,False,4000.00,True,7,156,490,18


In [8]:
dummies = pd.get_dummies(df)
scaled_dummies = pd.DataFrame(MinMaxScaler().fit_transform(dummies), index=dummies.index, columns=dummies.columns)
scaled_dummies

,odometer_value,engine_has_gas,engine_capacity,has_warranty,price_usd,is_exchangeable,number_of_photos,up_counter,duration_listed,age
0,0.190000,0.0,0.294872,0.0,0.217984,0.0,0.094118,0.006452,0.007168,0.116883
1,0.290000,0.0,0.358974,0.0,0.099982,1.0,0.129412,0.028495,0.037186,0.220779
2,0.402000,0.0,0.294872,0.0,0.055981,1.0,0.035294,0.038172,0.067652,0.233766
3,0.010000,0.0,0.358974,0.0,0.199964,1.0,0.094118,0.022043,0.038530,0.259740
4,0.280000,0.0,0.294872,0.0,0.042663,1.0,0.152941,0.003226,0.003136,0.233766
...,...,...,...,...,...,...,...,...,...,...
38526,0.290000,0.0,0.423077,0.0,0.054981,1.0,0.047059,0.045161,0.134857,0.246753
38527,0.321000,0.0,0.256410,0.0,0.095982,1.0,0.035294,0.010215,0.142025,0.194805
38528,0.777957,0.0,0.423077,0.0,0.085982,0.0,0.023529,0.033333,0.165323,0.246753
38529,0.020000,0.0,0.230769,0.0,0.079982,1.0,0.070588,0.083333,0.219534,0.233766


In [9]:
X_train, X_test, y_train, y_test = train_test_split(scaled_dummies.drop('price_usd', axis=1), scaled_dummies['price_usd'])

## Random Forest

In [10]:
rf = train_random_forest(X_train, y_train)
evaluate_model(rf, X_test, y_test)

{'rmse': 0.05302652316084133,
 'mae': 0.03168711392853584,
 'r2': 0.8285503755995923}

## Compares worse compared to the base numerical model